In [1]:
import pandas as pd
import csv
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

df=pd.read_csv('training.csv')

vectorizer = TfidfVectorizer()

#give each word a tf-idf value
train_features = vectorizer.fit_transform(df['article_words'])

print(train_features.shape)

y_train = df['topic']

(9500, 35822)


In [4]:
df_test=pd.read_csv('test.csv')

#give each word a tf-idf value
test_features = vectorizer.transform(df_test['article_words'])

print(test_features.shape)

y_test = df_test['topic']

(500, 35822)


In [5]:
from sklearn import linear_model
from sklearn.calibration import CalibratedClassifierCV

SGD_clf = linear_model.SGDClassifier(tol=1e-2, shuffle=False, learning_rate='constant', eta0=1)
SGD_clf.fit(train_features, y_train)

SGD_prediction_test = SGD_clf.predict(test_features)
print('test data accuracy: {}'.format(np.mean(SGD_prediction_test == y_test)))

SGD_prediction_train = SGD_clf.predict(train_features)
print('train data accuracy: {}'.format(np.mean(SGD_prediction_train == y_train)))

test data accuracy: 0.79
train data accuracy: 0.8709473684210526


Compute probablity results

In [6]:
clf_sigmoid = CalibratedClassifierCV(SGD_clf, cv='prefit', method='sigmoid')
clf_sigmoid.fit(train_features, y_train)

target_names = ["ARTS CULTURE ENTERTAINMENT", "BIOGRAPHIES PERSONALITIES PEOPLE", "DEFENCE", "DOMESTIC MARKETS", "FOREX MARKETS", "HEALTH", "IRRELEVANT", "MONEY MARKETS", "SCIENCE AND TECHNOLOGY", "SHARE LISTINGS", "SPORTS"] 


pd.DataFrame(clf_sigmoid.predict_proba(test_features)*100, columns=target_names)

,ARTS CULTURE ENTERTAINMENT,BIOGRAPHIES PERSONALITIES PEOPLE,DEFENCE,DOMESTIC MARKETS,FOREX MARKETS,HEALTH,IRRELEVANT,MONEY MARKETS,SCIENCE AND TECHNOLOGY,SHARE LISTINGS,SPORTS
0,0.093620,0.033603,0.058913,0.173214,0.997620,0.018953,85.437281,2.248963,0.178011,10.670874,0.088947
1,0.647307,1.366583,0.059379,0.105653,0.663601,0.086432,88.719436,7.541555,0.158454,0.082616,0.568984
2,0.117404,0.002549,0.005292,0.004464,42.896320,0.003790,38.331456,18.622325,0.001191,0.008135,0.007075
3,0.074109,0.122362,0.075923,0.197367,1.523439,0.299692,94.893835,2.723278,0.058017,0.011842,0.020137
4,0.058172,0.035008,0.011551,0.012002,7.881767,0.009248,90.953767,0.980968,0.004448,0.047455,0.005613
...,...,...,...,...,...,...,...,...,...,...,...
495,1.043038,0.030434,0.262995,0.131535,1.789098,0.058982,94.009779,2.397594,0.027380,0.220013,0.029151
496,0.034476,0.271029,0.052984,0.073905,0.923206,0.027916,0.774257,2.114824,0.045494,0.020376,95.661534
497,0.036214,0.035079,0.002686,0.004856,4.369922,0.002857,3.048495,92.491412,0.002019,0.000244,0.006216
498,0.027794,0.173190,0.030107,0.100686,1.630139,0.032084,84.270462,3.474938,0.043867,10.139430,0.077304


Classification report

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, SGD_prediction_test, target_names=target_names))

                                  precision    recall  f1-score   support

      ARTS CULTURE ENTERTAINMENT       0.33      0.67      0.44         3
BIOGRAPHIES PERSONALITIES PEOPLE       1.00      0.20      0.33        15
                         DEFENCE       0.89      0.62      0.73        13
                DOMESTIC MARKETS       0.50      0.50      0.50         2
                   FOREX MARKETS       0.58      0.54      0.56        48
                          HEALTH       0.75      0.64      0.69        14
                      IRRELEVANT       0.84      0.92      0.88       266
                   MONEY MARKETS       0.63      0.61      0.62        69
          SCIENCE AND TECHNOLOGY       0.00      0.00      0.00         3
                  SHARE LISTINGS       0.67      0.29      0.40         7
                          SPORTS       0.95      0.97      0.96        60

                        accuracy                           0.79       500
                       macro avg    